In [1]:
!pip install --ignore-install -q pyarrow
!pip install --ignore-install -q pyspark
!pip install --ignore-install -q findspark
!pip install --ignore-install -q yfinance

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Michael\\AppData\\Roaming\\Python\\Python312\\site-packages\\numpy\\core\\_multiarray_tests.cp312-win_amd64.pyd'
Check the permissions.



In [2]:
try:
  !git clone https://github.com/Michaelwwk/test.git
except:
  pass

Cloning into 'test'...


In [3]:
import yfinance as yf
import pyarrow.csv as pv
import pyarrow.parquet as pq
import findspark
findspark.init()

# import collections

from pyspark.sql import SparkSession
# from google.colab import drive
from pyspark.sql.functions import udf
from pyspark.sql.types import DateType
from datetime import datetime
from pyspark.sql.types import FloatType
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as func
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [4]:
# on Local

spark = SparkSession.builder.master("local").appName("Stock Ticker").config('spark.ui.port', '4050').getOrCreate()
# drive.mount('/content/drive')

someFile = "wordcount.txt"
stock_folder = "StockData"
stocks = spark.read.csv(stock_folder, header=True)
peopleDF = spark.read.json("people.json")

df_csv = spark.read.format('csv') \
                .option("inferSchema","true") \
                .option("header","true") \
                .option("sep",";") \
                .load("people.csv")

df_json = spark.read.format('json') \
                .option("inferSchema","true") \
                .option("header","true") \
                .option("sep",";") \
                .load("people.json")

# read and convert hdb resale price
hdb_table = pv.read_csv("resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv")
iris_data = spark.read.csv("iris-data.csv", header=True, inferSchema=True)
pq.write_table(hdb_table,'resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.parquet')
hdb_parquet = pq.ParquetFile('resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.parquet')

In [5]:
# # on Colab

# spark = SparkSession.builder.master("local").appName("Stock Ticker").config('spark.ui.port', '4050').getOrCreate()
# # drive.mount('/content/drive')

# someFile = "/content/test/wordcount.txt"
# stock_folder = "/content/test/StockData"
# stocks = spark.read.csv(stock_folder, header=True)
# peopleDF = spark.read.json("/content/test/people.json")

# df_csv = spark.read.format('csv') \
#                 .option("inferSchema","true") \
#                 .option("header","true") \
#                 .option("sep",";") \
#                 .load("/content/test/people.csv")

# df_json = spark.read.format('json') \
#                 .option("inferSchema","true") \
#                 .option("header","true") \
#                 .option("sep",";") \
#                 .load("/content/test/people.json")

# # read and convert hdb resale price
# hdb_table = pv.read_csv("/content/test/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv")
# iris_data = spark.read.csv("/content/drive/MyDrive/iris-data.csv", header=True, inferSchema=True)
# pq.write_table(hdb_table,'/content/test/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.parquet')
# hdb_parquet = pq.ParquetFile('/content/test/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.parquet')

In [6]:
# import collections
# spark = SparkSession.builder.master("local").appName("My App ").getOrCreate()

In [7]:
spark.sparkContext

<SparkContext master=local appName=Stock Ticker>

In [8]:
# the above file is under your pythonProject folder
spark = SparkSession.builder.appName("SimpleApp").getOrCreate()
print(spark.read.text(someFile).count())


20


In [9]:
# to read in data from a text file, first upload the data file into your google drive and then mount your google drive onto colab
# to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True)
# drive.mount('/content/drive', force_remount=True)

In [10]:
rdd = spark.sparkContext.parallelize((1, 2, 3, 4, 5))
# Traditional Python map(function, collection) (few MBs - GB fails)
# Scalabale map (Peta - support)
# iterablecollection.map(function) -> Object
# collect() Object to collection
print(tuple(rdd.map(lambda x: x * x).collect()))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 7) (Michael-ASUS-ZenBook executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark-3.4.2-bin-hadoop3\python\lib\pyspark.zip\pyspark\worker.py", line 683, in main
RuntimeError: Python in worker has different version 3.10 than that in driver 3.12, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:561)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:767)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:749)
	at org.apa

In [ ]:
print(rdd.filter(lambda x: x % 2 == 0).collect())  # Keeps even numbers


[2, 4]


In [ ]:
words = spark.sparkContext.parallelize(["hello world", "hi", "hello mars", "hello jupiter", "hello saturn"])
print(words.flatMap(lambda x: x.split(" ")).collect())

['hello', 'world', 'hi', 'hello', 'mars', 'hello', 'jupiter', 'hello', 'saturn']


In [ ]:
print(words.flatMap(lambda x: x.split(" ")).distinct().collect())

['hello', 'world', 'hi', 'mars', 'jupiter', 'saturn']


In [ ]:
tuples = spark.sparkContext.parallelize((1, 1, 2, 3, 3, 4))
print(tuples.distinct().collect())


[1, 2, 3, 4]


In [ ]:
print(tuples.distinct().collect())

[1, 2, 3, 4]


In [ ]:
print(tuples.count())

6


In [ ]:
first_three = tuples.take(3)
print(first_three)


[1, 1, 2]


In [ ]:
sum = tuples.reduce(lambda a, b: a + b)
print(sum)


14


In [ ]:
pprint()

Pretty printing has been turned ON


In [ ]:
rdd = spark.sparkContext.parallelize([(3, 6),(1, 2),(3, 4)])
grouped = rdd.groupByKey()
for key, values in grouped.collect():
    print(f"{key}: {tuple(values)}")
reduced = rdd.reduceByKey(lambda a, b: a + b)
print(reduced.collect())
sorted_rdd = rdd.sortByKey()
print(sorted_rdd.collect())

3: (6, 4)
1: (2,)
[(3, 10), (1, 2)]
[(1, 2), (3, 6), (3, 4)]


In [ ]:
# Create two RDDs
rdd1 = spark.sparkContext.parallelize([("Alice", 1), ("Bob", 2)])
rdd2 = spark.sparkContext.parallelize([("Charlie", 3), ("David", 4)])

# Perform the union operation
union_rdd = rdd1.union(rdd2)

# Collect and print the results
print(union_rdd.collect())



[('Alice', 1), ('Bob', 2), ('Charlie', 3), ('David', 4)]


In [ ]:

# Create two RDDs with common keys
rdd3 = spark.sparkContext.parallelize([("Alice", "Apple"), ("Bob", "Banana")])
rdd4 = spark.sparkContext.parallelize([("Alice", 1), ("Bob", 2)])

# Perform the join operation
join_rdd = rdd3.join(rdd4)

# Collect and print the results
print(join_rdd.collect())


[('Alice', ('Apple', 1)), ('Bob', ('Banana', 2))]


In [ ]:
# Create two RDDs
rdd5 = spark.sparkContext.parallelize([1, 2])
rdd6 = spark.sparkContext.parallelize(["a", "b"])

# Perform the cartesian operation
cross_rdd = rdd5.cartesian(rdd6)

# Collect and print the results
print(cross_rdd.collect())

[(1, 'a'), (1, 'b'), (2, 'a'), (2, 'b')]


In [ ]:
# # to read in data from a text file, first upload the data file into your google drive and then mount your google drive onto colab
# from google.colab import drive
# # to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True)
# drive.mount('/content/drive', force_remount=True)

In [ ]:
# # install pyspark using pip
# !pip install --ignore-install -q pyspark
# # install findspark using pip
# !pip install --ignore-install -q findspark

In [ ]:
# from pyspark.sql import SparkSession
# import yfinance as yf
# stock_folder = "/content/drive/MyDrive/data/StockData"
# # stock_prices_data.to_csv("aapl_stock_prices_data.csv")

# from pyspark.sql.functions import udf
# from pyspark.sql.types import DateType
# from datetime import datetime

# from pyspark.sql.types import FloatType
# from pyspark.sql.types import IntegerType
# import pyspark.sql.functions as func
# from pyspark.sql.window import Window

# ## Reading CSV data => Stocks
# stocks = spark.read.csv(stock_folder, header=True)

# # import collections
# spark = SparkSession.builder.master("local").appName("Stock Ticker").config('spark.ui.port', '4050').getOrCreate()

In [ ]:
# Imports

ticker = "AAPL"  # Replace with the ticker symbol of the company you want to analyze
start_date = "2020-01-01"
end_date = "2023-12-28"

stock_prices_data = yf.download(ticker, start=start_date, end=end_date)

stock_prices_data.to_csv("aapl_stock_prices_data.csv")

[*********************100%%**********************]  1 of 1 completed


In [ ]:
## Seeing Data => Dataframe
stocks.show(5)

+------+----------+----------+-------+--------+--------+--------+
|Ticker|      Date|Close/Last| Volume|    Open|    High|     Low|
+------+----------+----------+-------+--------+--------+--------+
| BRK-B|05/31/2023|  $321.08 |6175417|$321.12 |$322.41 |$319.39 |
| BRK-B|05/30/2023|  $322.19 |3232461|$321.86 |$322.47 |$319.00 |
| BRK-B|05/26/2023|  $320.60 |3229873|$320.44 |$322.63 |$319.67 |
| BRK-B|05/25/2023|  $319.02 |4251935|$320.56 |$320.56 |$317.71 |
| BRK-B|05/24/2023|  $320.20 |3075393|$322.71 |$323.00 |$319.56 |
+------+----------+----------+-------+--------+--------+--------+
only showing top 5 rows



In [ ]:
## Seeing Schema of the Data => Data Types in Dataframe
stocks.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)



In [ ]:
## Basic select operation => Select Ticker, Date and Close price
stocks.select("Ticker").show(3)

+------+
|Ticker|
+------+
| BRK-B|
| BRK-B|
| BRK-B|
+------+
only showing top 3 rows



In [ ]:
stocks.select(["Ticker", "Date", "Open"]).show(5)

+------+----------+--------+
|Ticker|      Date|    Open|
+------+----------+--------+
| BRK-B|05/31/2023|$321.12 |
| BRK-B|05/30/2023|$321.86 |
| BRK-B|05/26/2023|$320.44 |
| BRK-B|05/25/2023|$320.56 |
| BRK-B|05/24/2023|$322.71 |
+------+----------+--------+
only showing top 5 rows



In [ ]:
## Filtering Data => Select rows containing Microsoft Stock in last one month
stocks.filter(stocks.Ticker == "MSFT").show(10)

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|05/31/2023|  $328.39 |45950550|$332.29 |$335.94 |$327.33 |
|  MSFT|05/30/2023|  $331.21 |29503070|$335.23 |$335.74 |$330.52 |
|  MSFT|05/26/2023|  $332.89 |36630630|$324.02 |$333.40 |$323.88 |
|  MSFT|05/25/2023|  $325.92 |43301740|$323.24 |$326.90 |$320.00 |
|  MSFT|05/24/2023|  $313.85 |23384890|$314.73 |$316.50 |$312.61 |
|  MSFT|05/23/2023|  $315.26 |30797170|$320.03 |$322.72 |$315.25 |
|  MSFT|05/22/2023|  $321.18 |24115660|$318.60 |$322.59 |$318.01 |
|  MSFT|05/19/2023|  $318.34 |27546700|$316.74 |$318.75 |$316.37 |
|  MSFT|05/18/2023|  $318.52 |27275990|$314.53 |$319.04 |$313.72 |
|  MSFT|05/17/2023|  $314.00 |24315010|$312.29 |$314.43 |$310.74 |
+------+----------+----------+--------+--------+--------+--------+
only showing top 10 rows



In [ ]:
stocks.filter((stocks.Ticker == "MSFT") & (stocks.Date == "05/31/2023")).show()

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|05/31/2023|  $328.39 |45950550|$332.29 |$335.94 |$327.33 |
+------+----------+----------+--------+--------+--------+--------+



In [ ]:
stocks.filter(((stocks.Ticker == "MSFT") | (stocks.Ticker == "V")) & (stocks.Date == "05/31/2023")).show()

+------+----------+----------+--------+--------+--------+--------+
|Ticker|      Date|Close/Last|  Volume|    Open|    High|     Low|
+------+----------+----------+--------+--------+--------+--------+
|  MSFT|05/31/2023|  $328.39 |45950550|$332.29 |$335.94 |$327.33 |
|     V|05/31/2023|  $221.03 |20460620|$219.96 |$221.53 |$216.14 |
+------+----------+----------+--------+--------+--------+--------+



In [ ]:
stocks.filter((stocks.Ticker.isin(["MSFT", "QQQ", "SPY", "V", "TSLA"])) & (stocks.Date == "05/31/2023")).show()

+------+----------+----------+---------+--------+--------+--------+
|Ticker|      Date|Close/Last|   Volume|    Open|    High|     Low|
+------+----------+----------+---------+--------+--------+--------+
|  MSFT|05/31/2023|  $328.39 | 45950550|$332.29 |$335.94 |$327.33 |
|  TSLA|05/31/2023|  $203.93 |150711700|$199.78 |$203.95 |$195.12 |
|     V|05/31/2023|  $221.03 | 20460620|$219.96 |$221.53 |$216.14 |
|   SPY|05/31/2023|    417.85|110811800|  418.28|  419.22|  416.22|
|   QQQ|05/31/2023|    347.99| 65105380|  348.37|   350.6|  346.51|
+------+----------+----------+---------+--------+--------+--------+



In [ ]:
stocks.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)



In [ ]:
## User Defined Functions

date_parser = udf(lambda date: datetime.strptime(date,"%m/%d/%Y"), DateType())
stocks = stocks.withColumn("ParsedDate", date_parser(stocks.Date))
stocks.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- ParsedDate: date (nullable = true)



In [ ]:
def num_parser(value):
    if isinstance(value, str):
        return float(value.strip("$"))
    elif isinstance(value, int) or isinstance(value, float):
        return value
    else:
        return None

parser_number = udf(num_parser, FloatType())

In [ ]:
stocks = (stocks.withColumn("Open", parser_number(stocks.Open))
                .withColumn("Close", parser_number(stocks["Close/Last"]))
                .withColumn("Low", parser_number(stocks.Low))
                .withColumn("High", parser_number(stocks.High)))

In [ ]:
stocks.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- ParsedDate: date (nullable = true)
 |-- Close: float (nullable = true)



In [ ]:
parse_int = udf(lambda value: int(value), IntegerType())
## Changing the datatype of the column
stocks = stocks.withColumn("Volume", parse_int(stocks.Volume))

In [ ]:
stocks.printSchema()

root
 |-- Ticker: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Close/Last: string (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- ParsedDate: date (nullable = true)
 |-- Close: float (nullable = true)



In [ ]:
cleaned_stocks = stocks.select(["Ticker", "ParsedDate", "Volume", "Open", "Low", "High", "Close"])
cleaned_stocks.show(5)

+------+----------+-------+------+------+------+------+
|Ticker|ParsedDate| Volume|  Open|   Low|  High| Close|
+------+----------+-------+------+------+------+------+
| BRK-B|2023-05-31|6175417|321.12|319.39|322.41|321.08|
| BRK-B|2023-05-30|3232461|321.86| 319.0|322.47|322.19|
| BRK-B|2023-05-26|3229873|320.44|319.67|322.63| 320.6|
| BRK-B|2023-05-25|4251935|320.56|317.71|320.56|319.02|
| BRK-B|2023-05-24|3075393|322.71|319.56| 323.0| 320.2|
+------+----------+-------+------+------+------+------+
only showing top 5 rows



In [ ]:
## Calculating basic stastics about data => Calculate average stock price
cleaned_stocks.describe(["Volume", "Open", "Low", "High", "Close"]).show()

+-------+--------------------+------------------+------------------+------------------+------------------+
|summary|              Volume|              Open|               Low|              High|             Close|
+-------+--------------------+------------------+------------------+------------------+------------------+
|  count|               15108|             15108|             15108|             15108|             15108|
|   mean|5.1868408793685466E7|180.09656566181036| 177.9982781513109| 182.1253348687101| 180.1256089860054|
| stddev| 5.496484129953461E7|101.16125813324406|100.26590135955226|101.96625521621772|101.14891782168523|
|    min|              961133|             12.07|              11.8|             12.45|             11.93|
|    max|           914080943|            479.22|            476.06|            479.98|            477.71|
+-------+--------------------+------------------+------------------+------------------+------------------+



In [ ]:
## Calculate maximum stock price for various stocks
cleaned_stocks.groupBy("Ticker").max("Open").show(15)

+------+---------+
|Ticker|max(Open)|
+------+---------+
| BRK-B|   361.39|
|   TSM|   141.61|
|  AAPL|   182.63|
|  META|   381.68|
|  TSLA|   411.47|
|   QQQ|   405.57|
|     V|   250.05|
| GOOGL|   151.25|
|   SPY|   479.22|
|  AMZN|    187.2|
|  MSFT|   344.62|
|  NVDA|   405.95|
+------+---------+



In [ ]:

cleaned_stocks.groupBy("Ticker").max("Open").withColumnRenamed("max(Open)", "MaxStockPrice").show(15)

+------+-------------+
|Ticker|MaxStockPrice|
+------+-------------+
| BRK-B|       361.39|
|   TSM|       141.61|
|  AAPL|       182.63|
|  META|       381.68|
|  TSLA|       411.47|
|   QQQ|       405.57|
|     V|       250.05|
| GOOGL|       151.25|
|   SPY|       479.22|
|  AMZN|        187.2|
|  MSFT|       344.62|
|  NVDA|       405.95|
+------+-------------+



In [ ]:
cleaned_stocks.groupBy("Ticker").agg(func.max("Open").alias("MaxStockPrice")).show(15)

+------+-------------+
|Ticker|MaxStockPrice|
+------+-------------+
| BRK-B|       361.39|
|   TSM|       141.61|
|  AAPL|       182.63|
|  META|       381.68|
|  TSLA|       411.47|
|   QQQ|       405.57|
|     V|       250.05|
| GOOGL|       151.25|
|   SPY|       479.22|
|  AMZN|        187.2|
|  MSFT|       344.62|
|  NVDA|       405.95|
+------+-------------+



In [ ]:
cleaned_stocks.groupBy("Ticker").agg(
    func.max("Open").alias("MaxStockPrice"),
    func.sum("Volume").alias("TotalVolume")
).show(15)

+------+-------------+------------+
|Ticker|MaxStockPrice| TotalVolume|
+------+-------------+------------+
| BRK-B|       361.39|  5862401321|
|   TSM|       141.61| 12506470104|
|  AAPL|       182.63|139310061360|
|  META|       381.68| 30148848043|
|  TSLA|       411.47|171802975076|
|   QQQ|       405.57| 60437153773|
|     V|       250.05| 10410997871|
| GOOGL|       151.25| 43956560981|
|   SPY|       479.22|107925285300|
|  AMZN|        187.2|104503287430|
|  MSFT|       344.62| 37976660472|
|  NVDA|       405.95| 58787218324|
+------+-------------+------------+



In [ ]:
## Calculate maximum price of stocks each year => Basic date manipulation operation
cleaned_stocks = (cleaned_stocks.withColumn("Year", func.year(cleaned_stocks.ParsedDate))
                                .withColumn("Month", func.month(cleaned_stocks.ParsedDate))
                                .withColumn("Day", func.dayofmonth(cleaned_stocks.ParsedDate))
                                .withColumn("Week", func.weekofyear(cleaned_stocks.ParsedDate))
                 )
cleaned_stocks.show(10)

+------+----------+-------+------+------+------+------+----+-----+---+----+
|Ticker|ParsedDate| Volume|  Open|   Low|  High| Close|Year|Month|Day|Week|
+------+----------+-------+------+------+------+------+----+-----+---+----+
| BRK-B|2023-05-31|6175417|321.12|319.39|322.41|321.08|2023|    5| 31|  22|
| BRK-B|2023-05-30|3232461|321.86| 319.0|322.47|322.19|2023|    5| 30|  22|
| BRK-B|2023-05-26|3229873|320.44|319.67|322.63| 320.6|2023|    5| 26|  21|
| BRK-B|2023-05-25|4251935|320.56|317.71|320.56|319.02|2023|    5| 25|  21|
| BRK-B|2023-05-24|3075393|322.71|319.56| 323.0| 320.2|2023|    5| 24|  21|
| BRK-B|2023-05-23|4031342|328.19|322.97|329.27|323.11|2023|    5| 23|  21|
| BRK-B|2023-05-22|2763422|330.75|328.35|331.49|329.13|2023|    5| 22|  21|
| BRK-B|2023-05-19|4323538| 331.0|329.12|333.94|330.39|2023|    5| 19|  20|
| BRK-B|2023-05-18|2808329|326.87|325.85|329.98|329.76|2023|    5| 18|  20|
| BRK-B|2023-05-17|3047626|325.02|324.82|328.26|327.39|2023|    5| 17|  20|
+------+----

In [ ]:
## Calculate average stock price for stock each month
monthly = cleaned_stocks.groupBy(['Ticker', 'Year', 'Month']).agg(func.max("Open").alias("MonthHigh"), func.min("Open").alias("MonthLow"))
weekly = cleaned_stocks.groupBy(['Ticker', 'Year', 'Week']).agg(func.max("Open").alias("WeekHigh"), func.min("Open").alias("WeekLow"))
monthly.show()

+------+----+-----+---------+--------+
|Ticker|Year|Month|MonthHigh|MonthLow|
+------+----+-----+---------+--------+
| BRK-B|2022|   10|   297.98|  260.58|
|  META|2020|    6|   241.28|  209.75|
| BRK-B|2018|    9|   222.13|  209.21|
|  MSFT|2022|    6|    275.2|  243.86|
|  MSFT|2021|    2|   245.03|  230.01|
|     V|2020|    7|    200.0|   189.5|
|   SPY|2019|    6|   296.04|  275.31|
|   QQQ|2019|    6|   189.72|  171.98|
|   QQQ|2019|    2|    174.2|  166.73|
|  MSFT|2020|    1|   174.05|  157.08|
| BRK-B|2021|   10|   290.85|  273.02|
| BRK-B|2020|   10|   216.74|  200.03|
|  TSLA|2023|    4|   199.91|  152.64|
|  TSLA|2019|    4|    19.22|   15.72|
|  AMZN|2018|    5|    81.15|   81.15|
|     V|2020|    4|   181.78|  152.53|
| GOOGL|2018|    5|     54.1|    54.1|
|  NVDA|2021|    3|    139.0|   121.3|
|  NVDA|2019|    6|    41.25|   33.98|
|     V|2018|    7|   143.08|  131.96|
+------+----+-----+---------+--------+
only showing top 20 rows



In [ ]:
weekly.show()

+------+----+----+--------+-------+
|Ticker|Year|Week|WeekHigh|WeekLow|
+------+----+----+--------+-------+
| BRK-B|2022|  14|   352.0| 341.17|
| BRK-B|2022|  10|  326.59| 322.49|
| BRK-B|2021|  14|  264.22| 260.02|
|  META|2022|  43|  131.68|  97.98|
|  META|2020|   6|  212.51| 203.44|
|  TSLA|2022|  20|  255.72| 235.67|
|  TSLA|2020|  19|   52.92|  46.73|
|  TSLA|2020|  16|   51.49|  39.34|
|  TSLA|2018|  39|   20.86|  18.02|
| GOOGL|2021|  29|  130.43| 125.53|
|  NVDA|2020|  37|  129.89| 117.35|
|   SPY|2022|  42|  375.13| 364.01|
|   SPY|2021|  52|  477.93| 472.06|
|   QQQ|2022|  34|  320.28| 313.61|
|   QQQ|2022|  29|  306.43| 293.11|
|   QQQ|2021|  45|  399.16| 391.77|
| BRK-B|2018|  48|  217.23|  209.3|
|  MSFT|2022|   6|  309.87| 301.25|
|  MSFT|2021|   2|  218.47| 213.52|
|  META|2022|  40|  140.49| 136.76|
+------+----+----+--------+-------+
only showing top 20 rows



In [ ]:
weekly.withColumn("Spread", weekly['WeekHigh'] - weekly['WeekLow']).show()

+------+----+----+--------+-------+---------+
|Ticker|Year|Week|WeekHigh|WeekLow|   Spread|
+------+----+----+--------+-------+---------+
| BRK-B|2022|  14|   352.0| 341.17|10.829987|
| BRK-B|2022|  10|  326.59| 322.49| 4.100006|
| BRK-B|2021|  14|  264.22| 260.02| 4.200012|
|  META|2022|  43|  131.68|  97.98| 33.69999|
|  META|2020|   6|  212.51| 203.44| 9.069992|
|  TSLA|2022|  20|  255.72| 235.67|20.050003|
|  TSLA|2020|  19|   52.92|  46.73|6.1899986|
|  TSLA|2020|  16|   51.49|  39.34|12.150002|
|  TSLA|2018|  39|   20.86|  18.02|2.8400002|
| GOOGL|2021|  29|  130.43| 125.53| 4.899994|
|  NVDA|2020|  37|  129.89| 117.35|12.540001|
|   SPY|2022|  42|  375.13| 364.01|11.119995|
|   SPY|2021|  52|  477.93| 472.06| 5.869995|
|   QQQ|2022|  34|  320.28| 313.61|6.6700134|
|   QQQ|2022|  29|  306.43| 293.11|13.320007|
|   QQQ|2021|  45|  399.16| 391.77|7.3900146|
| BRK-B|2018|  48|  217.23|  209.3|7.9299927|
|  MSFT|2022|   6|  309.87| 301.25| 8.619995|
|  MSFT|2021|   2|  218.47| 213.52

In [ ]:
yearly = cleaned_stocks.groupBy(['Ticker', 'Year']).agg(func.max("Open").alias("YearlHigh"), func.min("Open").alias("YearlyLow"))
yearly.show()

+------+----+---------+---------+
|Ticker|Year|YearlHigh|YearlyLow|
+------+----+---------+---------+
|  NVDA|2020|   147.04|    50.02|
|     V|2022|   233.04|    175.0|
|  META|2020|   300.16|   139.75|
| BRK-B|2023|    331.0|   294.68|
|   TSM|2018|     45.0|    35.33|
|   TSM|2021|   141.61|    108.0|
|   SPY|2022|   479.22|  349.205|
|  MSFT|2019|   159.45|    99.55|
|  MSFT|2021|   344.62|   212.17|
| BRK-B|2018|    224.0|   185.43|
|  META|2021|   381.68|    247.9|
|  TSLA|2019|     29.0|    12.07|
|   QQQ|2020|   314.16|   170.92|
|  META|2018|   215.72|    123.1|
|  AAPL|2020|   138.05|    57.02|
|  MSFT|2020|   229.27|   137.01|
|  TSLA|2021|   411.47|   184.18|
|     V|2021|   250.05|    192.0|
|  NVDA|2018|    72.33|    31.62|
|   SPY|2020|   373.81|   228.19|
+------+----+---------+---------+
only showing top 20 rows



In [ ]:
# Joins
cleaned_stocks.join(yearly, (cleaned_stocks.Ticker==yearly.Ticker) & (cleaned_stocks.Year == yearly.Year),
                                      'inner'
                                     ).show()

+------+----------+-------+------+------+------+------+----+-----+---+----+------+----+---------+---------+
|Ticker|ParsedDate| Volume|  Open|   Low|  High| Close|Year|Month|Day|Week|Ticker|Year|YearlHigh|YearlyLow|
+------+----------+-------+------+------+------+------+----+-----+---+----+------+----+---------+---------+
| BRK-B|2023-05-31|6175417|321.12|319.39|322.41|321.08|2023|    5| 31|  22| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-30|3232461|321.86| 319.0|322.47|322.19|2023|    5| 30|  22| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-26|3229873|320.44|319.67|322.63| 320.6|2023|    5| 26|  21| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-25|4251935|320.56|317.71|320.56|319.02|2023|    5| 25|  21| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-24|3075393|322.71|319.56| 323.0| 320.2|2023|    5| 24|  21| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-23|4031342|328.19|322.97|329.27|323.11|2023|    5| 23|  21| BRK-B|2023|    331.0|   294.68|
| BRK-B|2023-05-22|2763422|3

In [ ]:
cleaned_stocks.join(yearly,
                    (cleaned_stocks.Ticker==yearly.Ticker) & (cleaned_stocks.Year == yearly.Year),
                    'inner'
                   ).drop(yearly.Year, yearly.Ticker).show()

+----------+-------+------+------+------+------+-----+---+----+------+----+---------+---------+
|ParsedDate| Volume|  Open|   Low|  High| Close|Month|Day|Week|Ticker|Year|YearlHigh|YearlyLow|
+----------+-------+------+------+------+------+-----+---+----+------+----+---------+---------+
|2023-05-31|6175417|321.12|319.39|322.41|321.08|    5| 31|  22| BRK-B|2023|    331.0|   294.68|
|2023-05-30|3232461|321.86| 319.0|322.47|322.19|    5| 30|  22| BRK-B|2023|    331.0|   294.68|
|2023-05-26|3229873|320.44|319.67|322.63| 320.6|    5| 26|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-25|4251935|320.56|317.71|320.56|319.02|    5| 25|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-24|3075393|322.71|319.56| 323.0| 320.2|    5| 24|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-23|4031342|328.19|322.97|329.27|323.11|    5| 23|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-22|2763422|330.75|328.35|331.49|329.13|    5| 22|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-19|4323538| 331.0|329.12|333.94

In [ ]:
historic_stocks = cleaned_stocks.join(yearly,
                    (cleaned_stocks.Ticker==yearly.Ticker) & (cleaned_stocks.Year == yearly.Year),
                    'inner'
                   ).drop(yearly.Year, yearly.Ticker)

In [ ]:
historic_stocks.show(5)

+----------+-------+------+------+------+------+-----+---+----+------+----+---------+---------+
|ParsedDate| Volume|  Open|   Low|  High| Close|Month|Day|Week|Ticker|Year|YearlHigh|YearlyLow|
+----------+-------+------+------+------+------+-----+---+----+------+----+---------+---------+
|2023-05-31|6175417|321.12|319.39|322.41|321.08|    5| 31|  22| BRK-B|2023|    331.0|   294.68|
|2023-05-30|3232461|321.86| 319.0|322.47|322.19|    5| 30|  22| BRK-B|2023|    331.0|   294.68|
|2023-05-26|3229873|320.44|319.67|322.63| 320.6|    5| 26|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-25|4251935|320.56|317.71|320.56|319.02|    5| 25|  21| BRK-B|2023|    331.0|   294.68|
|2023-05-24|3075393|322.71|319.56| 323.0| 320.2|    5| 24|  21| BRK-B|2023|    331.0|   294.68|
+----------+-------+------+------+------+------+-----+---+----+------+----+---------+---------+
only showing top 5 rows



In [ ]:
snapshot = cleaned_stocks.select(['Ticker', 'ParsedDate', 'Open'])
snapshot.show()

+------+----------+------+
|Ticker|ParsedDate|  Open|
+------+----------+------+
| BRK-B|2023-05-31|321.12|
| BRK-B|2023-05-30|321.86|
| BRK-B|2023-05-26|320.44|
| BRK-B|2023-05-25|320.56|
| BRK-B|2023-05-24|322.71|
| BRK-B|2023-05-23|328.19|
| BRK-B|2023-05-22|330.75|
| BRK-B|2023-05-19| 331.0|
| BRK-B|2023-05-18|326.87|
| BRK-B|2023-05-17|325.02|
| BRK-B|2023-05-16|322.46|
| BRK-B|2023-05-15|322.89|
| BRK-B|2023-05-12|323.82|
| BRK-B|2023-05-11| 321.0|
| BRK-B|2023-05-10|326.08|
| BRK-B|2023-05-09|324.87|
| BRK-B|2023-05-08|328.26|
| BRK-B|2023-05-05|323.36|
| BRK-B|2023-05-04|323.44|
| BRK-B|2023-05-03|327.13|
+------+----------+------+
only showing top 20 rows



In [ ]:
lag1Day = Window.partitionBy("Ticker").orderBy("ParsedDate")
snapshot.withColumn("PreviousOpen", func.lag("Open", 1).over(lag1Day)).show()


+------+----------+-----+------------+
|Ticker|ParsedDate| Open|PreviousOpen|
+------+----------+-----+------------+
|  AAPL|2018-05-31|46.81|        NULL|
|  AAPL|2018-06-01| 47.0|       46.81|
|  AAPL|2018-06-04|47.91|        47.0|
|  AAPL|2018-06-05|48.27|       47.91|
|  AAPL|2018-06-06|48.41|       48.27|
|  AAPL|2018-06-07|48.54|       48.41|
|  AAPL|2018-06-08|47.79|       48.54|
|  AAPL|2018-06-11|47.84|       47.79|
|  AAPL|2018-06-12|47.85|       47.84|
|  AAPL|2018-06-13|48.11|       47.85|
|  AAPL|2018-06-14|47.89|       48.11|
|  AAPL|2018-06-15|47.51|       47.89|
|  AAPL|2018-06-18|46.97|       47.51|
|  AAPL|2018-06-19|46.29|       46.97|
|  AAPL|2018-06-20|46.59|       46.29|
|  AAPL|2018-06-21|46.81|       46.59|
|  AAPL|2018-06-22|46.53|       46.81|
|  AAPL|2018-06-25|45.85|       46.53|
|  AAPL|2018-06-26|45.75|       45.85|
|  AAPL|2018-06-27|46.31|       45.75|
+------+----------+-----+------------+
only showing top 20 rows



In [ ]:

## Calculate moving average
movingAverage = Window.partitionBy("Ticker").orderBy("ParsedDate").rowsBetween(-50, 0)
(snapshot.withColumn("MA50", func.avg("Open").over(movingAverage))
         .withColumn("MA50", func.round("MA50", 2))).show()

+------+----------+-----+-----+
|Ticker|ParsedDate| Open| MA50|
+------+----------+-----+-----+
|  AAPL|2018-05-31|46.81|46.81|
|  AAPL|2018-06-01| 47.0|46.91|
|  AAPL|2018-06-04|47.91|47.24|
|  AAPL|2018-06-05|48.27| 47.5|
|  AAPL|2018-06-06|48.41|47.68|
|  AAPL|2018-06-07|48.54|47.82|
|  AAPL|2018-06-08|47.79|47.82|
|  AAPL|2018-06-11|47.84|47.82|
|  AAPL|2018-06-12|47.85|47.82|
|  AAPL|2018-06-13|48.11|47.85|
|  AAPL|2018-06-14|47.89|47.86|
|  AAPL|2018-06-15|47.51|47.83|
|  AAPL|2018-06-18|46.97|47.76|
|  AAPL|2018-06-19|46.29|47.66|
|  AAPL|2018-06-20|46.59|47.59|
|  AAPL|2018-06-21|46.81|47.54|
|  AAPL|2018-06-22|46.53|47.48|
|  AAPL|2018-06-25|45.85|47.39|
|  AAPL|2018-06-26|45.75| 47.3|
|  AAPL|2018-06-27|46.31|47.25|
+------+----------+-----+-----+
only showing top 20 rows



In [ ]:
maximumStock = Window.partitionBy("Ticker").orderBy(snapshot.Open.desc())
snapshot.withColumn("MaxOpen", func.row_number().over(maximumStock)).show()

+------+----------+------+-------+
|Ticker|ParsedDate|  Open|MaxOpen|
+------+----------+------+-------+
|  AAPL|2022-01-04|182.63|      1|
|  AAPL|2021-12-13|181.12|      2|
|  AAPL|2021-12-28|180.16|      3|
|  AAPL|2022-01-05|179.61|      4|
|  AAPL|2021-12-30|179.47|      5|
|  AAPL|2021-12-29|179.33|      6|
|  AAPL|2021-12-16|179.28|      7|
|  AAPL|2022-03-30|178.55|      8|
|  AAPL|2021-12-31|178.09|      9|
|  AAPL|2022-03-31|177.84|     10|
|  AAPL|2022-01-03|177.83|     11|
|  AAPL|2022-04-05| 177.5|     12|
|  AAPL|2023-05-31|177.33|     13|
|  AAPL|2021-12-27|177.09|     14|
|  AAPL|2023-05-30|176.96|     15|
|  AAPL|2022-03-29|176.69|     16|
|  AAPL|2023-05-19|176.39|     17|
|  AAPL|2022-01-12|176.12|     18|
|  AAPL|2022-02-09|176.05|     19|
|  AAPL|2021-12-23|175.85|     20|
+------+----------+------+-------+
only showing top 20 rows



In [ ]:
## Calculate top 5 highest close price for each stock in a year
snapshot.withColumn("MaxOpen", func.row_number().over(maximumStock)).filter("MaxOpen<=5").show()

+------+----------+------+-------+
|Ticker|ParsedDate|  Open|MaxOpen|
+------+----------+------+-------+
|  AAPL|2022-01-04|182.63|      1|
|  AAPL|2021-12-13|181.12|      2|
|  AAPL|2021-12-28|180.16|      3|
|  AAPL|2022-01-05|179.61|      4|
|  AAPL|2021-12-30|179.47|      5|
|  AMZN|2021-07-12| 187.2|      1|
|  AMZN|2021-07-09|186.13|      2|
|  AMZN|2021-07-07|185.87|      3|
|  AMZN|2021-11-19|185.63|      4|
|  AMZN|2021-07-14|185.44|      5|
| BRK-B|2022-03-29|361.39|      1|
| BRK-B|2022-03-28|360.59|      2|
| BRK-B|2022-03-31| 359.0|      3|
| BRK-B|2022-03-30|354.66|      4|
| BRK-B|2022-03-25| 353.9|      5|
| GOOGL|2022-02-02|151.25|      1|
| GOOGL|2021-11-19|149.98|      2|
| GOOGL|2021-11-08|149.83|      3|
| GOOGL|2021-11-22|149.33|      4|
| GOOGL|2021-11-09|149.23|      5|
+------+----------+------+-------+
only showing top 20 rows



In [ ]:
result = snapshot.withColumn("MaxOpen", func.row_number().over(maximumStock)).filter("MaxOpen<=5")
moving_avg = (snapshot.withColumn("MA50", func.avg("Open").over(movingAverage))
         .withColumn("MA50", func.round("MA50", 2)))
moving_avg.show()

+------+----------+-----+-----+
|Ticker|ParsedDate| Open| MA50|
+------+----------+-----+-----+
|  AAPL|2018-05-31|46.81|46.81|
|  AAPL|2018-06-01| 47.0|46.91|
|  AAPL|2018-06-04|47.91|47.24|
|  AAPL|2018-06-05|48.27| 47.5|
|  AAPL|2018-06-06|48.41|47.68|
|  AAPL|2018-06-07|48.54|47.82|
|  AAPL|2018-06-08|47.79|47.82|
|  AAPL|2018-06-11|47.84|47.82|
|  AAPL|2018-06-12|47.85|47.82|
|  AAPL|2018-06-13|48.11|47.85|
|  AAPL|2018-06-14|47.89|47.86|
|  AAPL|2018-06-15|47.51|47.83|
|  AAPL|2018-06-18|46.97|47.76|
|  AAPL|2018-06-19|46.29|47.66|
|  AAPL|2018-06-20|46.59|47.59|
|  AAPL|2018-06-21|46.81|47.54|
|  AAPL|2018-06-22|46.53|47.48|
|  AAPL|2018-06-25|45.85|47.39|
|  AAPL|2018-06-26|45.75| 47.3|
|  AAPL|2018-06-27|46.31|47.25|
+------+----------+-----+-----+
only showing top 20 rows



In [ ]:

# # #from pyspark import SparkConf,SparkContext
# # from pyspark.sql import SparkSession
# # import collections
# # spark = SparkSession.builder.master("local").appName("Ingestion").config('spark.ui.port', '4050').getOrCreate()

# df_csv = spark.read.format('csv') \
#                 .option("inferSchema","true") \
#                 .option("header","true") \
#                 .option("sep",";") \
#                 .load("/content/drive/MyDrive/data/DataFormat/people.csv")

# df_json = spark.read.format('json') \
#                 .option("inferSchema","true") \
#                 .option("header","true") \
#                 .option("sep",";") \
#                 .load("/content/drive/MyDrive/data/DataFormat/people.json")

# peopleDF = spark.read.json("/content/drive/MyDrive/data/DataFormat/people.json")

# import pyarrow.csv as pv
# import pyarrow.parquet as pq

# # read hdb resale price
# hdb_table = pv.read_csv("/content/drive/MyDrive/data/DataFormat/resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv")
# # convert the CSV file to a Parquet file
# pq.write_table(hdb_table,'resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.parquet')
# hdb_parquet = pq.ParquetFile('resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.parquet')


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Read CSV file people.csv
# Show result
df_csv.show()
# Print schema
df_csv.printSchema()


+-----+---+---------+
| name|age|      job|
+-----+---+---------+
|Jorge| 30|Developer|
|  Bob| 32|Developer|
+-----+---+---------+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)



In [ ]:
# # Write csv file
# df = spark.read.format('csv') \
#                 .option("inferSchema","true") \
#                 .option("header","true") \
#                 .option("sep",";") \
#                 .load("people.csv")


In [ ]:
# DataFrames can be saved as Parquet files, maintaining the schema information.
peopleDF.write.format("parquet").mode("overwrite").save("people.parquet")

In [ ]:
# Read in the Parquet file created above.
# Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a DataFrame.
parquetFile = spark.read.parquet("people.parquet")

# Parquet files can also be used to create a temporary view and then used in SQL statements.
parquetFile.createOrReplaceTempView("parquetFile")
teenagers = spark.sql("SELECT name FROM parquetFile WHERE age >= 13 AND age <= 19")
teenagers.show()


+------+
|  name|
+------+
|Justin|
+------+



In [ ]:
# inspect the parquet metadata
print(hdb_parquet.metadata)
# inspect the parquet row group metadata
print(hdb_parquet.metadata.row_group(0))
# inspect the column chunk metadata
print(hdb_parquet.metadata.row_group(0).column(9).statistics)

  created_by: parquet-cpp-arrow version 10.0.1
  num_columns: 10
  num_rows: 52203
  num_row_groups: 1
  format_version: 2.6
  serialized_size: 2079
  num_columns: 10
  num_rows: 52203
  total_byte_size: 431095
  has_min_max: True
  min: 195000.0
  max: 1088888.0
  null_count: 0
  distinct_count: 0
  num_values: 52203
  physical_type: DOUBLE
  logical_type: None
  converted_type (legacy): NONE


In [ ]:
# Assuming the target variable is "class" and other columns are features
feature_cols = iris_data.columns[:-1]

In [ ]:
# Convert string labels into numerical labels
indexer = StringIndexer(inputCol="class", outputCol="label")
iris_data = indexer.fit(iris_data).transform(iris_data)

In [ ]:
# Create a feature vector by assembling the feature columns
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(iris_data)

In [ ]:
# Split the data into training and testing sets
(training_data, testing_data) = data.randomSplit([0.8, 0.2], seed=123)

In [ ]:
# Customized parameters
num_trees = 10
max_depth = 5

In [ ]:
# Create and train a RandomForestClassifier with customized parameters
rf = RandomForestClassifier(
    labelCol="label",
    featuresCol="features",
    numTrees=num_trees,
    maxDepth=max_depth
)

In [ ]:
model = rf.fit(training_data)

In [ ]:
# Make predictions on the testing data
predictions = model.transform(testing_data)

In [ ]:
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

In [ ]:
# Print the accuracy
print("Accuracy: {:.2f}".format(accuracy))


Accuracy: 0.97


In [ ]:
# Show the feature importances
print("Feature Importances: ", model.featureImportances)

Feature Importances:  (4,[0,1,2,3],[0.10971043638291314,0.027259903033869593,0.5388647415001908,0.3241649190830265])


In [ ]:
# Stop the Spark session
spark.stop()